# Make Tables for Main Manuscript

Lauren Khoury  
July 30, 2025

In [ ]:
#| label: set up

library(dplyr) |> suppressMessages()
library(skimr)

In [ ]:
#| label: tbl-dictionary

data_tbl0 <- tibble(
  variable = c("b_x", "n_obs", "n_covs", "r_ycov", "p_good_covs", "r_cov"),
  description = c(
    "the population parameter for X",
    "number of observations in sample",
    "number of covariates",
    "correlation between Y and covariates",
    "percentage of \"good\" covariates",
    "correlation between \"good\" covariates"
  ),
  values = c("0, 0.3, 0.5",
             "50, 100, 150, 200, 300, 400", 
             "4, 8, 12, 16, 20", 
             "0.3, 0.5", 
             "0.25, 0.50, 0.75", 
             "0.3")
)

data_tbl0 |>
  kbl(col.names = c("Variable", "Description", "Values"), 
      escape = FALSE) |>
  kable_styling(bootstrap_options = c("striped", "hover"), 
                full_width = FALSE,
                font_size = 20) |>
  column_spec(1, width = "8em") |>
  column_spec(2, width = "15em") |>
  column_spec(3, width = "10em")

Variable,Description,Values
b_x,the population parameter for X,"0, 0.3, 0.5"
n_obs,number of observations in sample,"50, 100, 150, 200, 300, 400"
n_covs,number of covariates,"4, 8, 12, 16, 20"
r_ycov,correlation between Y and covariates,"0.3, 0.5"
p_good_covs,"percentage of ""good"" covariates","0.25, 0.50, 0.75"
r_cov,"correlation between ""good"" covariates",0.3
